In [ ]:
from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras("/kaggle/input/cropnet-mobilenetv3/models")

In [ ]:
image_size = 224
resize_scale = 1.5
image_resize_shape = int(resize_scale * image_size)
batch_size = 32

In [ ]:
import tf_keras as keras
from pathlib import Path
import tensorflow as tf

In [ ]:
val_transforms = [
    lambda img: tf.image.resize(
        img, (image_resize_shape, image_resize_shape)
    ),
    lambda img: tf.image.resize_with_crop_or_pad(
        img, target_height=image_size, target_width=image_size
    ),
    lambda img: img / 255.0,
]

def val_image_transforms(image):
    for fn in val_transforms:
        image = fn(image)
    return image

In [ ]:
from PIL import Image

def open_image(path):
    with Image.open(path) as image:
        image = keras.utils.img_to_array(image)
    return val_image_transforms(image)

In [ ]:
folder = Path("/kaggle/input/cassava-leaf-disease-classification/test_images")
images = [(path.name, open_image(path)) for path in folder.glob("*")]

In [ ]:
ids, inputs = map(list, zip(*images))

In [ ]:
input_data = tf.data.experimental.from_list(inputs).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for input in input_data:
    print(input_data)

In [ ]:
outputs = model.predict(input_data)[:, :-1]

In [ ]:
labels = tf.argmax(outputs, axis=-1).numpy()

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "image_id": ids,
    "label": labels
})

In [ ]:
df

In [ ]:
df.to_csv("submission.csv", index=False)